In [11]:
import sys
# can't relative import with sister directory
sys.path.insert(0, "..")
import ipywidgets
import pandas as pd
import cocpit
from ai2es import gui_setup

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
# USER CHANGES
year = 2017
time_of_day = 'night' # or 'day'
precip = 'no_precip'
precip_threshold = 0.1  # threshold of rain gauge output be to be considered precipitating

In [21]:
df, folder_dest = gui_setup.read_parquet(year, time_of_day, precip_threshold, precip)
df = gui_setup.shuffle_df(df)
gui_setup.make_folders(folder_dest)
#gui_setup.show_new_images()

In [22]:
g = cocpit.gui_label.GUI(df['path'], folder_dest, df['precip_accum_1min [mm]'])
g.display_image()
g.make_buttons()
g.align_buttons()
box_layout = ipywidgets.Layout(
        display="flex", flex_flow="column", align_items="center", width="100%"
    )
image_box = ipywidgets.HBox([g.center])
display(ipywidgets.VBox([image_box, g.label_btns, g.undo_btn], layout=box_layout))